In [105]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import sys
import iprompt.data
from transformers import AutoTokenizer, OPTForCausalLM, AutoModelForCausalLM
# from iprompt.data import TASKS_GALACTICA
import transformers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
results_dir = '/home/chansingh/iprompt/experiments/results'

In [107]:
tox_results = {
    fname.replace('.pkl', ''): pkl.load(open(oj(results_dir, fname), 'rb'))
    for fname in os.listdir(results_dir)
    if fname.endswith('.pkl')
    and fname.startswith('tox')
}

In [108]:
def contains_tox(x: list):
    return 'tox' in ''.join(x).lower()

def reciprocal_rank_tox(x: list):
    for i, y in enumerate(x):
        if 'tox' in y.lower():
            return 1 / (i + 1)
    return 0


r = {}
for k in tox_results:
    l = tox_results[k]['prefixes']
    r[k] = [contains_tox(l), contains_tox(l[:1]), reciprocal_rank_tox(l)]
    
df = pd.DataFrame.from_dict(r)
df.index = ['Any correct', 'Top-prompt correctness', 'MRR']
df_baseline = df[[k for k in df.columns if 'baseline' in k]]
df_iprompt = df[[k for k in df.columns if not 'baseline' in k]]

In [109]:
df_baseline

,tox21_baseline_100_3,tox21_baseline_100_7,tox21_baseline_100_8,tox21_baseline_100_16,tox21_baseline_100_9,tox21_baseline_100_17,tox21_baseline_100_15,tox21_baseline_100_5,tox21_baseline_100_11,tox21_baseline_100_1,tox21_baseline_100_18,tox21_baseline_100_12,tox21_baseline_100_6,tox21_baseline_100_14,tox21_baseline_100_4,tox21_baseline_100_10,tox21_baseline_100_19,tox21_baseline_100_13,tox21_baseline_100_2
Any correct,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Top-prompt correctness,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
MRR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [110]:
scores_iprompt = df_iprompt.mean(axis=1).round(2).astype(str) + \
    ' $\pm$ ' + \
    (df_iprompt.std(axis=1) /
     np.sqrt(df_iprompt.shape[1])).round(2).astype(str)


In [111]:
scores_baseline = df_baseline.mean(axis=1)

In [118]:
d = pd.concat((scores_iprompt, scores_baseline), axis=1)
d.columns = ['iPrompt', 'Null data']
d = d.loc[['MRR', 'Top-prompt correctness']]
print(d.to_latex(escape=False))

\begin{tabular}{llr}
\toprule
{} &          iPrompt &  Null data \\
\midrule
MRR                    &  0.83 $\pm$ 0.04 &        0.0 \\
Top-prompt correctness &  0.67 $\pm$ 0.08 &        0.0 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3454514/1765034884.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d.to_latex(escape=False))


In [119]:
df

,tox21_1_100_3,tox21_10_100_3,tox21_6_100_2,tox21_baseline_100_3,tox21_0_100_1,tox21_5_100_1,tox21_6_100_1,tox21_7_100_2,tox21_1_100_2,tox21_8_100_3,...,tox21_baseline_100_4,tox21_baseline_100_10,tox21_4_100_2,tox21_10_100_2,tox21_0_100_3,tox21_6_100_3,tox21_baseline_100_19,tox21_baseline_100_13,tox21_baseline_100_2,tox21_11_100_3
Any correct,True,True,True,False,True,True,True,True,True,True,...,False,False,True,True,True,True,False,False,False,True
Top-prompt correctness,False,False,True,False,True,True,True,True,True,False,...,False,False,True,True,False,False,False,False,False,False
MRR,0.5,0.5,1.0,0,1.0,1.0,1.0,1.0,1.0,0.5,...,0,0,1.0,1.0,0.5,0.5,0,0,0,0.5


In [120]:
for k in tox_results:
    print(tox_results[k]['prefixes'])

[' not in any dataset but answered', ' toxic, and Otherwise answer NO', ' toxic, and Otherwise answer no', ' an experimental positive but  no', ' in ToxList; no', ' in ToxList; otherwise', ' in ToxList; Otherwise', ' active from both datasets, i', ' classified as an ED; NO', ' found in DrugBank as Active', ' in category “Aromatic compounds', ' toxic, and Otherwise answer as', ' not in any dataset but T', ' not in any dataset but can', ' toxic, and answer no otherwise', ' found in DrugBank as an', ' not in any dataset but has', ' in dataset for all three ', ' soluble in water, and Not', ' soluble in water, and answer', ' an experimental positive but "', ' mutagenic and  no otherwise?', ' "Not mutagenic in Am', ' soluble in water, and no', ' "Not mutagenic in bacteria', ' toxic in at least one assay', ' "Not mutagenic in Salmonella', ' an explosive in terms of its']
[' not in any dataset but answered', ' toxic, and Otherwise answer NO', ' toxic, and Otherwise answer no', ' an experimenta